# Génération condtionnée (Seq2Seq) avec des RNNs et de l'attention

Dans le TP précédent, nous avons utilisé des RNNs pour générer du texte "libre" - ou bien conditionné par
le début de la séquence. Pour certaines tâches, comme par exemple la traduction ou la création
de légendes pour les images, il peut être intéressant de traiter de manière
différente la représentation des données en entrées et en sortie.

De plus, afin d'améliorer les performance des modèles, les RNNs peuvent utiliser une "mémoire" - dans
notre cas, il s'agit du texte en entrée. Cette idée est reprise dans les transformers que nous 
verrons dans le module suivant.

Dans cette partie, nous allons introduire deux nouveautés par rapport aux RNNs du TP précédent :

1. Nous allons utiliser un encodeur et un décodeur (seq2seq) avec des paramètres distincts
1. Nous allons utiliser un mécanisme d'attention

Les prochaines cellules permettent de charger et préparer les données

In [1]:
import os 
import sys
from typing import Tuple, Any, List, Union
import shutil
from torch.utils.tensorboard import SummaryWriter
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm.autonotebook import tqdm
from pathlib import Path

cachepath = os.path.expanduser('~/.local/data')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BASEPATH = Path("xp/seq2seq")
TB_PATH =  BASEPATH / "logs"
TB_PATH.mkdir(parents=True, exist_ok=True)

print(f"tensorboard --logdir {Path(TB_PATH).absolute()}")

tensorboard --logdir /Users/vguigue/Documents/Cours/Agro-IODAA/deep/notebooks/xp/seq2seq/logs


Nous allons utiliser le même jeu de données que dans le carnet précédent, mais en utilisant cette fois-ci les deux textes (document et résumé).

In [2]:
from datasets import load_dataset, load_metric

# On prend juste 10% de la validation pour aller plus vite
raw_datasets = load_dataset("xsum", split={"train": "train[:10%]", "validation": "validation[:5%]", "test": "validation[5%:]"})

# Dans le cadre du résumé, nous allons utiliser la métrique "rouge"
rouge = load_metric("rouge")

Found cached dataset xsum (/Users/vguigue/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
100%|██████████| 3/3 [00:00<00:00, 92.17it/s]
/var/folders/_h/xz4nr0h53dj3x3tygxjnzl540000gn/T/ipykernel_75062/62614406.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [3]:
print(rouge.inputs_description)


Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Returns:
    rouge1: rouge_1 (precision, recall, f1),
    rouge2: rouge_2 (precision, recall, f1),
    rougeL: rouge_l (precision, recall, f1),
    rouge

Plutôt que d'utiliser un vocabulaire entraîné sur les textes en apprentissage, nous allons utiliser ici un vocabulaire plus
large qui a été utilisé pour BERT.

In [4]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', bos_token="<bos>", eos_token="<eos>")

ModuleNotFoundError: No module named 'transformers'

In [ ]:
batch = ["<bos> This is the first document <eos>", "<bos> followed by the next one <eos>", "<bos> and the final text is here <eos>"]
r = tokenizer(batch,  truncation=True, add_special_tokens=False, return_token_type_ids=False, padding=True, return_tensors="pt")

print(r)

[" ".join(tokenizer.convert_ids_to_tokens(row)) for row in r["input_ids"]]

In [ ]:
def getdata(batch, what: str, device):
    """Fonction utilitaire pour réduire la taille des données en fonction du batch"""

    r = tokenizer([f"<bos> {t} <eos>" for t in batch[what]],  truncation=True, add_special_tokens=False, return_token_type_ids=False, padding=True, return_tensors="pt", max_length=512)
    # Renvoie dans le format RNN (temps en premier)
    return (r["input_ids"].T).to(device).contiguous()

# Exemple
loader = DataLoader(raw_datasets["train"], batch_size=2)
input_ids = getdata(next(iter(loader)), "summary", device)
input_ids, input_ids.shape

# <span style="background: green; padding: 3px; color: white">Exercice 1 : implémenter un seq2seq</span>

La cellule suivante permet de définir:

- `RNNBase` qui est le prototype qui sera utilisé par tous vos RNNs (encodeurs et décodeurs)
- `Seq2Seq` qui est un modèle qui permet de regrouper encodeur, décodeur et classifieur (logits de la distribution multinomiale sur les tokens)
- `train_seq2seq` qui permet d'apprendre un modèle `Seq2Seq`

In [ ]:

class RNNBase(nn.Module):
    """Cette classe sert de base pour tous vos modèles récurrents"""

    def __init__(self):
        super().__init__()

    def forward(self, x: torch.LongTensor, h_0=None, *, encoder_outputs=None, encoder_embeddings=None) -> Tuple[nn.Module, nn.Module, Any]:
        """Méthode principale pour les réseaux récurrents

        Les paramètres `encoder_*` serviront pour l'exercice 2

        Args:
            x (torch.LongTensor): Un tenseur contenant un batch de séquences sous forme d'ID de tokens (temps x batch) 
            h_0 (Any, optional): État initial à utiliser.
            encoder_outputs (torch.Tensor, optional): Les sorties de l'encodeur
            encoder_embeddings (torch.Tensor, optional): Les entrées de l'encodeur

        Returns:
            Tuple[nn.Module, nn.Module, Any]: Renvoie un tuple (embeddings, sorties du RNN, état final)
        """
        raise NotImplementedError()

class Seq2Seq(nn.Module):
    """Modèle Seq2Seq générique"""

    def __init__(self, name: str, encoder: nn.Module, decoder: nn.Module, classifier: nn.Module):
        """Initialise le modèle seq2seq

        Args:
            name (str): Le nom du modèle (pour tensorboard)
            encoder (nn.Module): Un RNN qui encode
            decoder (nn.Module): Un RNN qui décode
            classifier (nn.Module): Le classifieur
        """
        super().__init__()
        self.name = name
        self.encoder = encoder
        self.decoder = decoder
        self.classifier = classifier

    def forward(self, source_input_ids, target_input_ids):
        encoder_embeddings, encoder_outputs, hidden = self.encoder(source_input_ids)    # encodage => in, out, hidden_state
        _, output, hidden = self.decoder(target_input_ids, hidden, encoder_embeddings=encoder_embeddings, encoder_outputs=encoder_outputs)
                                                                                        # decodage => out, hidden
        return self.classifier(output), hidden, encoder_embeddings, encoder_outputs     # etat caché, embedding, sortie de l'encodeur

    def decoder_step(self, inputs, hidden, encoder_embeddings, encoder_outputs):
        _, output, hidden = self.decoder(inputs, hidden, encoder_outputs=encoder_outputs, encoder_embeddings=encoder_embeddings)
        return self.classifier(output), hidden

def generate(tokenizer, model: Seq2Seq, document: Union[str, List[str]], maxlength=50):
    """Génère une suite de tokens en utilisant la distribution de probabilité du modèle"""

    if isinstance(document, str):
        document = [document]

    with torch.no_grad():
        toks = tokenizer(document, return_tensors="pt", return_length=True, padding=True)
        
        x = toks["input_ids"].T.contiguous().to(device)

        # Séequences générées
        generated = [[] for _ in range(len(document))]
        lengths = [maxlength for _ in range(len(document))]

        bos = torch.LongTensor([[tokenizer.bos_token_id]]).tile(1, len(document)).to(device)
        y_t, s_t, encoder_embeddings, encoder_outputs = model(x, bos) # application du modèle

        for length in range(maxlength):
            w_t = torch.distributions.categorical.Categorical(logits=y_t[-1]).sample()

            w_t_cpu = w_t.cpu().numpy()
            for ix, (g, w) in enumerate(zip(generated, w_t_cpu)):
                g.append(int(w))
                if w == tokenizer.eos_token_id:
                    lengths[ix] = min(lengths[ix], length)


            y_t, s_t = model.decoder_step(w_t.unsqueeze(0), s_t, encoder_embeddings, encoder_outputs)

        return [tokenizer.decode(s[:lengths[ix]]) for ix, s in enumerate(generated)]


In [ ]:
TRAIN_BATCHSIZE = 128
TEST_BATCHSIZE = 128

def computeloss(batch, model, loss):
    """Calcule le coût du modèle sur un batch, ainsi que des métriques"""
    source_input_ids = getdata(batch, "document", device)
    target_input_ids = getdata(batch, "summary", device)
    yhat, *args = model(source_input_ids, target_input_ids[:-1])
    predicted, reference = yhat.view(-1, yhat.shape[2]), target_input_ids[1:].view(-1)
    return loss(predicted, reference)

def train_seq2seq(model: Seq2Seq, epochs: int, datasets, *, val_steps=1):
    """Entraînement des modèles
    
    Args:
        model (Seq2Seq): le modèle à entraîner
        epochs (int): le nombre d'époques d'entraînement
        val_steps (int, optional): le nombre d'époques entre chaque calcul de performance sur le jeu de validation
    """
    print(f"Training {model.name}")
    
    # On nettoie le rep. de log
    tbpath = f"{TB_PATH}/{model.name}"
    shutil.rmtree(tbpath, ignore_errors = True)
    writer = SummaryWriter(tbpath)
    
    optim = torch.optim.Adam(model.parameters(), lr=1e-4)
    model = model.to(device)

    train_loader = DataLoader(datasets["train"], TRAIN_BATCHSIZE, shuffle=True)
    test_loader = DataLoader(datasets["test"], TEST_BATCHSIZE, shuffle=False)
    loss = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_type_id)
    
    for epoch in tqdm(range(epochs)):
        cumloss, count =  0, 0
        model.train()
        for ix, batch in enumerate(train_loader):
            optim.zero_grad()
            l = computeloss(batch, model, loss)
            l.backward()
            optim.step()
            batchlen = len(batch["document"])
            cumloss += l.item() * batchlen
            count += batchlen

        writer.add_scalar('loss/train', cumloss/count, epoch)

        if epoch % val_steps == 0:
            model.eval()
            with torch.no_grad():
                cumloss, count = 0, 0
                for batch in test_loader:
                    l = computeloss(batch, model, loss)
                    batchlen = len(batch["document"])
                    
                    # Compute metrics
                    predictions = generate(tokenizer, model, batch["document"])
                    rouge.add_batch(predictions=predictions, references=batch["summary"])

                    cumloss += l * batchlen
                    count += batchlen
    
                for key, value in rouge.compute().items():
                    writer.add_scalar(f"{key}/test", value.mid.fmeasure, epoch)
                writer.add_scalar(f'loss/test', cumloss/count, epoch)

On peut maintenant reprendre le code du LSTM vu en 4.1 et l'adapter pour la tâche en respectant le prototype 
donné par `RNNBase` - pour l'instant, ignorez `encoder_outputs` et `encoder_embeddings`, ils seront utiles dans
la suite.

In [ ]:
# Reprendre le code du RNN et l'adapter

class LSTM(RNNBase):
#  TODO 

#  Maintenant, créez le model Seq2Seq en utilisant deux RNNs (enc+dec)
#  TODO 

# On regarde la génération (cela doit être totalement aléatoire pour l'instant...)
print(raw_datasets["test"][5]["document"][:400])
print("--->")
print(generate(tokenizer, model, raw_datasets["test"][5]["document"]))

# Maintenant, on peut entraîner notre modèle
# (model est un Seq2Seq)
train_seq2seq(model, 50, raw_datasets)

On peut maintenant voir les séquences générées en utilisant la méthode `generate` adaptée aux nouvelles sorties

In [ ]:
print(raw_datasets["test"][5]["document"][:400])
print("--->")
print(generate(tokenizer, model, raw_datasets["test"][5]["document"]))

# <span style="background: green; padding: 3px; color: white">Exercice 2 : Ajouter de l'attention</span>

Nous allons maintenant faire un pas de plus vers les transformers... en utilisant un mécanisme d'attention.

Pour faire cela, nous allons tout d'abord calculer une attention sur les sorties de l'encodeur $o_{1\ldots N}$ (tenseur temps x batch x dim. espace latent), et utiliser une combinaison des embeddings des entrées $x_{1\ldots M}$ (tenseur temps x batch x dim. embeddings). 

Étant donné les sorties du décodeur, $z_{1\ldots M}$, l'attention est calculée de la manière suivante :

1. On calcule les "clefs" $k_{1\ldots N}$ en utilisant une transformation linéaire des sorties de l'encodeur (dimension $d$ arbitraire)
1. On calcule les "questions" $q_{1\ldots M}$ en utilisant une transformation linéaire des sorties du décodeur (même dimension $d$ que les clefs)
1. On calcule le produit scalaire de chaque clef $k_{i,j}$ (vecteur de dimension $d$) avec chaque question $q_{k, j}$ (pour un échantillon $j$) puis normalisons avec `softmax` pour obtenir une distribution de probabilité conditionnelle que le token $k$ du décodeur utilise le token $i$ de l'encodeur $p_j(i|k)$ : 
   $$ p_j(k|i) \propto \exp\left( k_{i,j} \cdot q_{k, j} \right)$$
1. On modifie la sortie du décodeur en ajoutant une combinaison convexe des embeddings de l'encodeur (cela permet d'utiliser des mots du vocabulaire utilisée dans le texte source plus facilement) :
   $$ z^{\prime}_{i, j} = z_{i, j} + \sum_{k=1}^{N} p_j(k|i) v(x_k) $$ 
   où $v$ est une fonction de transformation (vous pouvez utiliser l'identité si la dimension des sorties du RNN est la même que celle des embeddings)

Créez une classe spécifique pour le décodeur et entraînez votre nouveau modèle, puis visualisez les résultats - vous devriez obtenir une diminution du coût en entraînement (et en validation), ainsi qu'une qualité un peu meilleure des sorties.

In [ ]:
# Créer un nouveau décodeur qui utilise de l'attention sur l'encodeur

# correction précédente
class LSTM(RNNBase):
    def __init__(self, vocab_size, embeddings_dim, hidden_dim):
        super().__init__()
    
        self.embeddings = nn.Embedding(vocab_size, embeddings_dim)
        self.rnn = nn.LSTM(embeddings_dim, hidden_dim)        
    
    def forward(self, x, h_0=None, *, encoder_outputs=None, encoder_embeddings=None):
        x = self.embeddings(x)
        output, hidden = self.rnn(x, h_0)
        return x, output, hidden


class LSTMWithAttention(nn.Module):
#  TODO 


vocab_size = len(tokenizer.get_vocab())

embeddings = nn.Embedding(vocab_size, 100)
encoder = LSTM(embeddings, 100)
decoder = LSTMWithAttention(embeddings, 100)
classifier = nn.Linear(100, vocab_size)

model_att = Seq2Seq("lstm-att", encoder, decoder, classifier)
train_seq2seq(model_att, 50, raw_datasets)


In [ ]:
# Test du modèle sur un exemple

d = raw_datasets["test"][5]["document"]
print(d)
print("--->")
print(generate(tokenizer, model_att, d))

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###